In [ ]:
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 969.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, RandomSampler
from transformers import T5Model, T5ForConditionalGeneration, T5TokenizerFast
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Import custom module

In [ ]:
%cd /content/drive/MyDrive/ITMO/src
%ls

/content/drive/MyDrive/ITMO/src
medqadataset.py  __pycache__/  trainer.py


In [ ]:
from medqadataset import MedQADataset
import trainer
#from trainer import Trainer

In [ ]:
# # For debugging
# import importlib

# importlib.reload(trainer)

In [ ]:
MODEL_NAME = "cointegrated/rut5-base"    # pre-trained model (first iteration)
#MODEL_NAME = "/content/drive/MyDrive/ITMO/checkpoints/best_model/MedQAModel3"
#MODEL_NAME = "/content/drive/MyDrive/ITMO/checkpoints/best_model/MedQAModel4"
#TOKENIZER = T5TokenizerFast.from_pretrained(MODEL_NAME, local_files_only=True)
TOKENIZER = T5TokenizerFast.from_pretrained(MODEL_NAME)
# Set up the T5 model
MODEL = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
# Set up the optimizer
OPTIMIZER = Adam(MODEL.parameters(), lr=0.0001)
Q_LEN = 512    # Question Length
T_LEN = 128    # Target Length
BATCH_SIZE = 4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [ ]:
DEVICE

'cuda'

### Read Data

In [ ]:
PATH = '/content/drive/MyDrive/ITMO'

data = pd.read_csv(PATH + '/data/MedQuAD_context_answer_sum_concat.csv', index_col=0)
data.index = [i for i in range(len(data))]

data

,question,context,answer,answer_start
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...,9.0
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...,9.0
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю...",2551.0
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...,4573.0
4,что такое синдром кноблоха?,"синдром кноблоха — редкое заболевание, характ...","синдром кноблоха — редкое заболевание, характе...",9.0
...,...,...,...,...
2210,есть ли у вас информация об измерении темпера...,оставьте термометр во рту на 3 минуты или пока...,"если температура тела может быть высока, то эт...",NaN
2211,что такое дефицит аденозинмонофосфатдезаминазы?,у многих больных дефицит амфдезаминазы не вызы...,"ученые считают, что дефицит амфдезаминазы може...",NaN
2212,сколько людей страдают от дефицита аденозинмо...,у многих больных дефицит амфдезаминазы не вызы...,амфдезаминазы является одним из наиболее распр...,NaN
2213,"каковы генетические изменения, связанные с де...",у многих больных дефицит амфдезаминазы не вызы...,человек с дефицитом амфдезаминазы страдает от ...,NaN


### Example

In [ ]:
sample = data.iloc[0]
encoding = TOKENIZER(
    sample["question"],
    sample["context"],
    max_length=512,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors='pt'
)

answer_encoding = TOKENIZER(
    sample["answer"],
    max_length=32,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

labels = answer_encoding["input_ids"]

output = MODEL(
    input_ids=encoding["input_ids"],
    attention_mask=encoding["attention_mask"],
    labels=labels
)

In [ ]:
len(labels[0])

32

In [ ]:
labels = [
    TOKENIZER.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    for input_id in answer_encoding["input_ids"]
]


In [ ]:
" ".join(labels)

'кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волос'

In [ ]:
sample["answer"]

'кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волосы и во многих случаях повышают риск потенциально опасных для жизни проблем с сердцем. у людей с такими заболеваниями волосы необычно грубые, сухие, тонкие и туго завитые. в некоторых случаях волосы также редкие. шерстистая текстура волос обычно поражает только волосы на коже головы и присутствует с рождения. начиная с раннего возраста, у пораженных людей также развивается ладонно-подошвенная кератодермия — состояние, при котором кожа на ладонях рук и подошвах ног становится толстой, чешуйчатой \u200b\u200bи мозолистой. кардиомиопатия, заболевание сердечной мышцы, представляет собой опасную для жизни проблему со здоровьем, которая может развиться у людей с кератодермией и пушистыми волосами. в отличие от других особенностей этого состояния, признаки и симптомы кардиомиопатии могут не проявляться до подросткового возраста или позже. осложнения кардиомиопатии могут включать наруш

In [ ]:
labels

['кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волос']

In [ ]:
output.loss

tensor(9.9832, grad_fn=<NllLossBackward0>)

In [ ]:
sample_q = data.iloc[0]["question"]
sample_a = data.iloc[0]["answer"]

sample_encoding = TOKENIZER(sample_q, sample_a)

In [ ]:
sample_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
preds = [
    TOKENIZER.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    for input_id in sample_encoding["input_ids"]
]

In [ ]:
" ".join(preds)

'что тако е  кера то дерм ия с пуш исты ми волос ами ?   кера то дерм ия с пуш исты ми волос ами представля ет с обой групп у связ анных состояни й ,  которые пора жают кож у  и волос ы  и во мног их случа ях по выш ают риск потенци ально опас ных  для жизни проблем с серд цем . у люд ей с так ими заболевани ями волос ы не обычн о  груб ые , сух ие , тон кие  и ту го зави тые . в не которы х случа ях волос ы так же ред кие . шер сти стая текст ура волос  обычн о пора жает  только волос ы на кож е голов ы  и присут ствует с  рождени я . начина я с ран него возраст а , у по ражен ных люд ей так же развива ется лад онно - по дош вен ная  кера то дерм ия — состояни е , при котор ом кожа на  ладо нях рук  и по дош вах  ног станов ится тол стой ,  чеш уй чат ой  и моз оли стой . кардио ми опати я , заболевани е  сердеч ной мыш цы , представля ет с обой опас ную  для жизни проблем у со здоровь ем , котор ая  может раз виться у люд ей с  кера то дерм ией  и пуш исты ми волос ами . в от личие о

### Dataloaders

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.02, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Train shape: {train_data.shape}")
print(f"Validation shape: {val_data.shape}")
print(f"Test shape: {test_data.shape}")

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = MedQADataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(OPTIMIZER, patience=5, verbose=True)

Train shape: (1736, 4)
Validation shape: (434, 4)
Test shape: (45, 4)


### Train and validation loop

In [ ]:
N_EPOCHS = 100
STATE_DICT_PATH = '/content/drive/MyDrive/ITMO/checkpoints/best_model/MedQuAD_context_answer_sum_concat'

trainer_module = trainer.Trainer(
    MODEL,
    N_EPOCHS,
    STATE_DICT_PATH,
    train_loader,
    val_loader,
    OPTIMIZER,
    DEVICE,
    TOKENIZER,
    Q_LEN,
    T_LEN,
    scheduler
)

In [ ]:
trainer_module.fit()

Validation batches: 100%|██████████| 109/109 [00:30<00:00,  3.58it/s]


Epoch: 0		Train loss: 7.4267514630946145		Validation loss: 4.708329861317206


Validation batches: 100%|██████████| 109/109 [00:30<00:00,  3.60it/s]


Epoch: 1		Train loss: 4.350602991569976		Validation loss: 1.8094480939961355


Validation batches: 100%|██████████| 109/109 [00:30<00:00,  3.59it/s]


...

Validation batches: 100%|██████████| 109/109 [00:30<00:00,  3.62it/s]


Epoch: 32		Train loss: 0.46154506466767753		Validation loss: 0.15998144402519004


Validation batches: 100%|██████████| 109/109 [00:30<00:00,  3.61it/s]


Epoch: 33		Train loss: 0.4370050531927891		Validation loss: 0.14455707135413765


Training batches:  70%|███████   | 305/434 [03:57<01:40,  1.28it/s]

### Evaluate the results

In [ ]:
test_sample = test_data.iloc[3]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"
test_answer = f"Ответ: {test_sample['answer']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question,
   test_answer,
   print_text=True
)

pred

In [ ]:
# Calculate metrics on test
bleu_avg, rouge_avg = trainer_module.calc_avg_metrics(test_data)
print("Test:")
print("BLEU:", bleu_avg)
print("RougeL:", rouge_avg)

Test:
BLEU: 0.13342170575668585
RougeL: 0.08695652173913043


In [ ]:
# Plot loss
# General loss with previous steps of fine-tuning
trainer_module.plot_loss()

### Inference

In [ ]:
test_sample = test_data.iloc[3]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  что вызывает боль в пенисе? также называется: боль пенис
Контекст: боль в половом члене это любая боль или дискомфорт в половом члене. причины причины могут включать в себя: камни в мочевом пузыре. укусы человека или насекомых. рак полового члена. непроходящая эрекция приапизм. генитальный герпес. инфицированные волосяные фолликулы. инфицированный протез полового члена. инфекция под крайней плотью у необрезанных мужчин баланит. воспаление предстательной железы простатиттравма болезнь пейрони синдром рейтерасерповидноклеточная анемиясифилисуретрит, вызванный хламидиями или гонореей инфекция мочевого пузыря тромб в вене полового члена перелом полового члена уход на дому как лечить боль в половом члене дома зависит от ее причины. поговорите со своим врачом о лечении. пакеты со льдом могут помочь облегчить боль. не прикладывайте лед непосредственно к пенису. вместо этого поместите лед в полотенце или мочалку и приложите завернутый лед, чтобы облегчить боль. если боль в половом чле

In [ ]:
test_sample = test_data.iloc[15]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  что такое путаница? также называется: дезориентация; мышление неясное; мысли затуманенное
Контекст: замешательство это неспособность думать так ясно и быстро, как обычно. вы можете чувствовать дезориентацию и испытывать трудности с концентрацией внимания, запоминанием и принятием решений. соображения замешательство может нарастать быстро или медленно с течением времени, в зависимости от причины. во многих случаях спутанность сознания длится недолго и проходит. в других случаях это навсегда и неизлечимо. это может быть связано с бредом или деменцией. спутанность сознания чаще встречается у пожилых людей и часто возникает во время пребывания в больнице. некоторые люди с растерянностью могут вести себя странно или необычно или вести себя агрессивно. причины спутанность сознания может быть вызвана различными проблемами со здоровьем, например :алкогольная или наркотическая интоксикация опухоль головного мозга травма головы или травма головы сотрясение мозгалихорадкадисбаланс жидкос

In [ ]:
test_sample = test_data.iloc[22]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  каковы симптомы дивертикулеза? также называется: дивертикула дивертикулез; дивертикулит дивертикулез; дивертикулярная болезнь дивертикулез; желудочнокишечное кровотечение дивертикулез; желудочнокишечное кровотечение дивертикулез; желудочнокишечное кровотечение дивертикулез
Контекст: дивертикулез возникает, когда на внутренней стенке кишечника образуются маленькие выпуклые мешочки или мешочки. эти мешочки называются дивертикулами. чаще всего эти мешочки образуются в толстой кишке толстой кишке. они также могут возникать в тощей кишке и тонкой кишке, хотя это встречается реже. причины дивертикулез реже встречается у людей в возрасте 40 лет и моложе. это чаще встречается у пожилых людей. около половины американцев старше 60 лет страдают этим заболеванием. у большинства людей это случается к 80 годам. никто точно не знает, что вызывает образование этих мешочков. в течение многих лет считалось, что диета с низким содержанием клетчатки может сыграть свою роль. недостаток клетчатки м

In [ ]:
test_sample = test_data.iloc[9]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  что вызывает абсцесс?
Контекст: абсцесс это скопление гноя в любой части тела. в большинстве случаев область вокруг абсцесса опухает и воспаляется. причины абсцессы возникают, когда участок ткани заражается, и иммунная система организма пытается бороться с ним и сдерживать его. лейкоциты лейкоциты перемещаются через стенки кровеносных сосудов в область инфекции и собираются в поврежденной ткани. в ходе этого процесса образуется гной. гной это скопление жидкости, живых и мертвых лейкоцитов, мертвых тканей, бактерий или других посторонних веществ. абсцессы могут образовываться практически в любой части тела. кожа, под кожей и зубы являются наиболее распространенными местами. абсцессы могут быть вызваны бактериями, паразитами и инородными веществами. абсцессы на коже легко увидеть. они красные, приподнятые и болезненные. абсцессы в других частях тела могут быть не видны, но они могут вызывать повреждение органов. некоторые типы и локализации абсцессов включают: абдоминальный абсц